# dp100_23 データドリフトの監視

トレーニングと推論の間にあるデータプロファイルのこの変化は*データドリフト*と呼ばれ、  
運用環境で使用される予測モデルにとって重大な問題になる可能性がある。

そのため、時間の経過に伴うデータドリフトを監視し、必要に応じてモデルを再トレーニングして予測精度を維持できるようにすることが重要。

## データドリフトモニターの作成

データセットを使用したデータドリフトの監視をサポートしており、  
データセット内の新しい特徴量をキャプチャし、モデルのトレーニングに使用したデータセットと比較できる。

### データセットを比較してデータドリフトを監視する

モデルがトレーニングされた後も、組織が新しいデータを収集し続けることは一般的。  
増え続ける新しいデータのコレクションを元のトレーニングデータと定期的に比較し、  
モデルの精度に影響する可能性のあるデータの傾向の変化を特定することができる。

登録済みのデータセットを使用してデータドリフトを監視するには、次の2つのデータセットを登録する必要がある。

- *ベースライン"データセット
    - 通常は元のトレーニングデータ
- 時間間隔に基づいてベースラインと比較される"ターゲット"データセット
    - このデータセットには、比較する各特徴の列と、データドリフト率を測定するためのタイムスタンプ列が必要
    
> 注 : デプロイされたサービスを構成して、推論のためにモデルに送信された新しいデータを収集できる。  
このデータはAzure Blob Storageに保存され、データドリフトの監視対象のデータセットとして使用できる。

これらのデータセットを作成した後、*データセットモニター*を定義してデータドリフトを検出し、  
指定されたしきい値を誤差率が越えた場合にアラートをトリガーできる。

次のサンプルコードに示すように、AzureMLスタジオのビジュアルインターフェイスを使用するか、  
SDKの**DataDriftDetector**クラスを使用してデータセットモニターを作成できる。

```
from azureml.datadrift import DataDriftDetector

monitor = DataDriftDetector.create_from_datasets(workspace=ws,
                                                 name='dataset-drift-detector',
                                                 baseline_data_set=train_ds,
                                                 target_data_set=new_data_ds,
                                                 compute_target='aml-cluster',
                                                 frequency='Week',
                                                 feature_list=['age','height', 'bmi'],
                                                 latency=24)
```

データセットモニターを作成した後、以下のように*backfill*を実行すると、  
ベースラインデータセットをターゲットデータセットの既存のデータとすぐに比較できる。

```
import datetime as dt

backfill = monitor.backfill( dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())
```

## アラートのスケジュール設定

データモニターを定義するときに、その実行スケジュールを指定する。  
さらに、データドリフト率のしきい値と、このしきい値を超えた場合に通知するためのオペレータのメールアドレスを指定できる。

### データドリフトモニターのスケジュールを構成する

データドリフトの監視は、スケジュールが設定された**frequency**で比較を実行し、データドリフトメトリックを計算することで機能する。  
**Day**、**Week**、または**Month**ごとに実行するスケジュールを定義できる。

データセットモニターの場合、**latency**を指定して、新しいデータを収集し、ターゲットデータセットに追加できる時間数を示すことができる。
デプロイされたモデルのデータドリフトモニターの場合、データドリフトの実行を開始するタイミングを示す**schedule_start**の時間値を指定できる。

### アラートを構成する

データドリフトを測定するには、特徴量の値の統計的分布の時間経過に伴う変化を計算した*magnitude*を使用する。  
重大なデータドリフトを示す可能性がある大きな変化がないか監視する必要がある。

通知を受けたいデータドリフトの規模の**しきい値**を定義し、メールに依るアラート通知を構成することができる。  
以下のコードは、データドリフトモニターを毎週実行し、誤差の大きさが0.3を超える場合にアラートを送信するようにスケジュールを設定している。

```
alert_email = AlertConfiguration('data_scientists@contoso.com')
monitor = DataDriftDetector.create_from_datasets(ws, 'dataset-drift-detector', 
                                                 baseline_data_set, target_data_set,
                                                 compute_target=cpu_cluster,
                                                 frequency='Week', latency=2,
                                                 drift_threshold=.3,
                                                 alert_configuration=alert_email)
```

## 演習　データドリフトの監視

### 必要パッケージについて

In [1]:
!pip show azureml-datadrift

Name: azureml-datadrift
Version: 1.28.0
Summary: Azure Machine Learning datadrift
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: scipy, jsonpickle, scikit-learn, matplotlib, azureml-pipeline-core, azureml-core, azureml-dataset-runtime, azureml-telemetry, lightgbm, pandas, pyspark, msrest, numpy
Required-by: 


### ワークスペースへの接続

In [2]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with 20210613


### ベースラインデータセットの作成

データセットのデータドリフトを監視するためには、ベースラインデータセット(通常はモデルのトレーニングに使用したデータセット)を登録し、  
将来収集されるデータとの比較のポイントとして使用する必要がある。

In [3]:
from azureml.core import Datastore, Dataset


# Upload the baseline data
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-baseline',
                       overwrite=True, 
                       show_progress=True)

# Create and register the baseline dataset
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

Uploading an estimated of 2 files
Uploading ./data/diabetes.csv
Uploaded ./data/diabetes.csv, 1 files out of an estimated total of 2
Uploading ./data/diabetes2.csv
Uploaded ./data/diabetes2.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Registering baseline dataset...
Baseline dataset registered!


### ターゲットデータセットの作成

時間の経過とともに、ベースラインの訓練データセットと同じ特徴を持つ新しいデータを収集することができる。  
この新しいデータをベースラインデータと比較するには、データドリフトを分析したい特徴量を含むターゲットデータセットを定義する必要がある。  
タイムスタンプは、データセット自体のフィールド化、データの保存に使用されたフォルダとファイル名のパターンから得られるものがある。

In [4]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# Load the smaller of the two data files
data = pd.read_csv('data/diabetes2.csv')

# We'll generate data for the past 6 weeks
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # Get the date X weeks ago
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modify data to ceate some drift
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # Save the file with the date encoded in the filename
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# Upload the files
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# Use the folder partition format to define a dataset with a 'date' timestamp column
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# Register the target dataset
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

Generating simulated data...
Uploading an estimated of 6 files
Uploading data/diabetes_2021-05-12.csv
Uploaded data/diabetes_2021-05-12.csv, 1 files out of an estimated total of 6
Uploading data/diabetes_2021-05-19.csv
Uploaded data/diabetes_2021-05-19.csv, 2 files out of an estimated total of 6
Uploading data/diabetes_2021-05-26.csv
Uploaded data/diabetes_2021-05-26.csv, 3 files out of an estimated total of 6
Uploading data/diabetes_2021-06-02.csv
Uploaded data/diabetes_2021-06-02.csv, 4 files out of an estimated total of 6
Uploading data/diabetes_2021-06-09.csv
Uploaded data/diabetes_2021-06-09.csv, 5 files out of an estimated total of 6
Uploading data/diabetes_2021-06-16.csv
Uploaded data/diabetes_2021-06-16.csv, 6 files out of an estimated total of 6
Uploaded 6 files
Registering target dataset...
Target dataset registered!


### データドリフトモニターの作成

データドリフトモニターは、定期的またはオンデマンドで実行され、ベースラインデータセットとターゲットデータセットを比較する。  
※ターゲットデータセットには、時間の経過とともに新しいデータが追加される。

#### コンピューティングターゲットの作成

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "msl-20210613b"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


#### データドリフトモニターの定義

データドリフトを監視する特徴量、監視プロセスの実行に使用するコンピューティングターゲット名、データを比較する頻度、  
アラートが発生するデータドリフトのしきい値、データ収集のための待ち時間を指定する。

In [6]:
from azureml.datadrift import DataDriftDetector

# set up feature list
features = ['Pregnancies', 'Age', 'BMI']

# set up data drift detector
monitor = DataDriftDetector.create_from_datasets(ws, 'mslearn-diabates-drift', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

{'_workspace': Workspace.create(name='20210613', subscription_id='153404fd-72ab-4092-b50e-de490c5509fc', resource_group='20210613'), '_frequency': 'Week', '_schedule_start': None, '_schedule_id': None, '_interval': 1, '_state': 'Disabled', '_alert_config': None, '_type': 'DatasetBased', '_id': '6c10f0a1-8963-437c-95a7-cd7581c0ec57', '_model_name': None, '_model_version': 0, '_services': None, '_compute_target_name': 'msl-20210613b', '_drift_threshold': 0.3, '_baseline_dataset_id': 'e6f56144-90b4-4ee6-90c9-003825cfe4cd', '_target_dataset_id': '6dc1504e-756b-4cd2-81dd-527263e5a697', '_feature_list': ['Pregnancies', 'Age', 'BMI'], '_latency': 24, '_name': 'mslearn-diabates-drift', '_latest_run_time': None, '_client': <azureml.datadrift._restclient.datadrift_client.DataDriftClient object at 0x7f2d986b7860>, '_logger': <_TelemetryLoggerContextAdapter azureml.datadrift._logging._telemetry_logger.azureml.datadrift.datadriftdetector (DEBUG)>}

### データドリフトモニターのバックフィル(埋戻し)

ベースラインデータセットとターゲットデータセットを使ってモニターをバックフィルし、オリジナルのベースラインデータとターゲットデータの間の  
データドリフトを分析することができる。

> 注 : バックフィル分析を行うためには、コンピューティングターゲットを起動する必要があるので、  
実行に時間がかかる場合がある。

In [7]:
from azureml.widgets import RunDetails

backfill = monitor.backfill(dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-diabates-drift-Monitor-Runs_1623863168511',
 'target': 'msl-20210613b',
 'status': 'Completed',
 'startTimeUtc': '2021-06-16T17:16:11.266838Z',
 'endTimeUtc': '2021-06-16T17:21:23.481811Z',
 'warnings': [{'source': 'datadrift',
   'message': 'target dataset id:6dc1504e-756b-4cd2-81dd-527263e5a697 do not contain sufficient amount of data after timestamp filteringMinimum needed: 50 rows.Skipping calculation for time slice 2021-05-02 00:00:00 to 2021-05-09 00:00:00.'}],
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'c9f8a4e3-334a-4a85-82a6-4316506b3599',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'e6f56144-90b4-4ee6-90c9-003825cfe4cd'}, 'consumptionDetails': {'type': 'Reference'}}, {'dataset': {'id': '6dc1504e-756b-4cd2-81dd-527263e5a697'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition'

### データドリフトの解析

以下のコードを使用して、バックフィル実行で収集されたポイントインタイムのデータドリフトを調べることもできる。

In [8]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

start_date 2021-05-02
end_date 2021-06-20
frequency Week
Datadrift percentage {'days_from_start': [7, 14, 21, 28, 35, 42], 'drift_percentage': [74.19152901127207, 87.23985219136877, 91.74192122865539, 94.96492628559955, 97.58354951107833, 99.23199438682525]}


AzureMLスタジオでデータドリフトメトリックを可視化することもできる。

> データドリフトの監視の詳細URL : https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets  
> 公開されているサービスからデータを収集して、データドリフト監視の対象データセットとして使う場合のURL : https://docs.microsoft.com/azure/machine-learning/how-to-enable-data-collection

## 知識チェック

1. 昨年収集されたデータを含むデータセットを使用してモデルをトレーニングしました。  
今年は、新しいデータを収集する予定です。 モデルのパフォーマンスに影響する可能性のある、変化するデータの傾向を追跡したいと考えています。  
何をする必要がありますか?

    - 既存のトレーニング データセットの新しいバージョンで新しいデータを収集し、両方のデータセットをプロファイルします。
    - 別のデータセットで新しいデータを収集し、トレーニング データセットをベースラインとして、  
    新しいデータセットをターゲットとして使用するデータ ドリフト モニターを作成します。
    - トレーニング データセットを、元のトレーニング データと新しいデータの両方を含む新しいデータセットに置き換えます。


2. あなたはデータ ドリフト モニターを作成しています。 データ分布に大きな変化が検出された場合、  
データ サイエンス チームに自動的に通知したいと考えています。 どうすればよいでしょうか。

    - AlertConfiguration を定義し、drift_threshold 値を設定します。
    - データ サイエンティストが新しいデータを確認する時間を確保できるように、データ ドリフト モニターの待機時間を設定します。
    - データ サイエンス チームのメール アドレスをタグとして含め、トレーニング データセットをモデルに登録します。

↓解答

1. 別のデータセットで新しいデータを収集し、トレーニング データセットをベースラインとして、  
新しいデータセットをターゲットとして使用するデータ ドリフト モニターを作成します。

    - 変化するデータの傾向を追跡するために、トレーニング データをベースラインとして、  
    新しいデータをターゲットとして使用するデータ ドリフト モニターを作成します。


2. AlertConfiguration を定義し、drift_threshold 値を設定します。

    - データ ドリフトについてオペレーターに通知するには、通知先のメール アドレスと、  
    通知をトリガーする変化のレベルを定義する誤差のしきい値を指定した AlertConfiguration を作成します。